In [22]:
import pandas as pd
import numpy as np

import statsmodels.stats.multitest as smm
import scipy
from statsmodels.stats.weightstats import *

In [96]:
def write_answer(filename='filename.txt', answer='NaN'):
    if (not filename.endswith('.txt')):
        filename = 'filename.txt'
    with open(filename, 'w') as file:
        file.write(str(answer))

# Завантаження даних

In [2]:
df = pd.read_csv('gene_high_throughput_sequencing.csv')

# Перегляд сирих даних та основні статистики

In [3]:
df.head()

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,STT5427_Breast_023_normal,normal,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,STT5430_Breast_002_normal,normal,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,STT5439_Breast_003_normal,normal,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,STT5441_Breast_004_normal,normal,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72 entries, 0 to 71
Columns: 15750 entries, Patient_id to EIF1AY
dtypes: float64(15748), object(2)
memory usage: 8.7+ MB


In [14]:
df.describe()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
count,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,...,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000,72.000000
mean,2.463922,7.100958,19.056151,15.618688,23.537120,11.303466,8.921396,63.270147,53.906324,70.678573,...,5.563444,2.192029,1.967424,2.181360,1.729308,1.980733,16.834075,1.825827,2.287380,1.756827
std,1.413687,4.164703,9.287753,4.664146,4.475294,4.055122,3.270713,13.792140,45.635498,21.326237,...,2.389789,2.454218,2.047129,2.542652,1.507149,2.230157,7.126802,1.902716,3.185571,1.623151
min,0.833898,1.231043,4.941211,6.395270,8.520713,2.066576,1.364917,42.544976,24.616767,42.631422,...,1.144240,0.853957,0.853957,0.853957,0.833898,0.833898,4.675683,0.833898,0.853957,0.833898
25%,1.284642,3.892403,13.423946,12.574596,21.151624,8.712898,6.213396,53.093627,34.009687,54.202100,...,3.805875,1.042783,1.011350,1.011350,1.003337,1.003337,11.186633,1.003337,1.011350,1.003337
50%,2.104677,6.315551,16.734855,14.586150,23.477006,11.172590,8.703397,63.230911,39.554954,65.510651,...,5.617824,1.339507,1.273861,1.298543,1.252527,1.252527,16.514389,1.252527,1.273861,1.252527
75%,3.345067,8.916399,21.884325,17.695678,26.471909,13.568625,11.139240,69.880705,50.295756,79.427207,...,7.100942,1.815754,1.647255,1.677971,1.625364,1.611299,21.849385,1.607345,1.647255,1.611299
max,7.364879,20.006038,60.584449,29.659104,34.110743,27.441093,17.121366,132.144503,327.590426,128.606260,...,11.227770,11.913855,10.883310,11.692697,10.392539,10.640754,49.295538,9.919132,17.278985,9.333904


# Розбиття сирих даних на групи

In [7]:
normal_group = df[df['Diagnosis'] == 'normal'].drop(['Diagnosis', 'Patient_id'], axis=1)
early_group = df[df['Diagnosis'] == 'early neoplasia'].drop(['Diagnosis', 'Patient_id'], axis=1)
cancer_group = df[df['Diagnosis'] == 'cancer'].drop(['Diagnosis', 'Patient_id'], axis=1)

# Основні статистики здорової групи

In [10]:
normal_group.head()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
0,1.257614,2.408148,13.368622,9.494779,20.880435,12.722017,9.494779,54.349694,26.329928,57.553312,...,4.761250,1.257614,1.257614,1.257614,1.257614,1.257614,23.268694,1.257614,1.257614,1.257614
1,4.567931,16.602734,42.477752,25.562376,23.221137,11.622386,14.330573,72.445474,39.140813,119.572567,...,6.871902,1.815112,1.815112,1.815112,1.815112,1.815112,10.427023,1.815112,1.815112,1.815112
2,2.077597,3.978294,12.863214,13.728915,14.543176,14.141907,6.232790,57.011005,48.144685,55.579752,...,7.096343,2.077597,2.077597,2.077597,2.077597,2.077597,22.344226,2.077597,2.077597,2.077597
3,2.066576,8.520713,14.466035,7.823932,8.520713,2.066576,10.870009,53.292034,27.076621,54.454714,...,5.200770,2.066576,2.066576,2.066576,2.066576,2.066576,49.295538,2.066576,2.066576,2.066576
4,2.613616,3.434965,12.682222,10.543189,26.688686,12.484822,1.364917,67.140393,35.896701,48.134682,...,11.227770,1.364917,1.364917,1.364917,1.364917,1.364917,23.627911,1.364917,1.364917,1.364917


In [11]:
normal_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24 entries, 0 to 23
Columns: 15748 entries, LOC643837 to EIF1AY
dtypes: float64(15748)
memory usage: 2.9 MB


In [12]:
normal_group.describe()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
count,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,...,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000,24.000000
mean,2.681277,4.368497,15.159566,15.374351,21.459886,9.374537,8.105567,58.080871,34.983861,62.870812,...,6.242498,2.104486,1.662368,1.842830,1.525062,1.701654,17.106405,1.576004,1.960442,1.568210
std,1.508486,3.045372,6.829965,5.068511,4.396089,3.428871,3.261537,9.481097,6.845557,18.211312,...,2.586175,2.084905,1.194909,1.750036,0.902641,1.412672,8.596529,1.125119,1.967174,1.090322
min,0.938061,1.231043,4.941211,7.823932,8.520713,2.066576,1.364917,42.544976,24.934607,47.227688,...,1.144240,0.853957,0.853957,0.853957,0.853957,0.853957,6.833581,0.853957,0.853957,0.853957
25%,1.426379,2.931811,12.281513,11.459379,19.922107,7.445387,6.114602,51.634329,30.576534,50.767527,...,4.582607,1.074431,1.074431,1.074431,1.074431,1.074431,11.186633,1.074431,1.074431,1.074431
50%,2.556835,3.774399,14.300119,13.982747,22.355593,8.843000,7.617484,56.331258,34.062096,55.659464,...,5.827847,1.298543,1.270305,1.270305,1.244328,1.244328,16.271728,1.244328,1.270305,1.244328
75%,3.526431,4.457341,16.843958,17.796930,24.030828,10.733951,10.891322,66.512938,39.347884,70.519051,...,8.182003,1.877978,1.689081,1.689081,1.517398,1.517398,20.838521,1.517398,1.877978,1.517398
max,7.364879,16.602734,42.477752,26.031425,27.090263,19.314861,14.330573,75.034709,48.144685,119.572567,...,11.227770,9.391555,6.405211,8.328369,5.182588,6.586955,49.295538,6.405211,9.267712,6.218157


# Основні статистики групи з захворенням на ранній стадії

In [15]:
early_group.head()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
24,2.516305,11.430887,18.506550,13.969049,20.957007,10.374992,8.414330,68.513944,37.622379,62.319268,...,4.488498,1.314098,1.314098,1.314098,3.307073,1.314098,25.059902,1.314098,1.314098,1.314098
25,1.937270,9.686352,23.541357,15.295034,18.815807,11.128772,9.155183,67.951908,51.729112,62.425015,...,3.709591,1.937270,1.937270,1.937270,1.937270,1.937270,26.349727,1.937270,1.937270,1.937270
26,1.405858,15.119783,17.985461,17.237294,21.824785,4.801919,5.796501,67.064975,33.298362,58.412228,...,7.029290,1.405858,1.405858,1.405858,1.405858,1.405858,29.254009,1.405858,1.405858,1.405858
27,2.131757,8.789458,12.731187,6.395270,19.185811,14.922297,4.082003,52.028259,43.670673,57.028643,...,9.453726,2.131757,2.131757,2.131757,2.131757,2.131757,23.189490,2.131757,2.131757,2.131757
28,2.421766,7.830416,18.283935,15.229320,29.234970,15.779540,8.098199,50.224123,42.944858,68.447282,...,5.976863,1.264726,1.264726,1.264726,1.264726,1.264726,19.849890,1.264726,1.264726,1.264726


In [16]:
early_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 24 to 48
Columns: 15748 entries, LOC643837 to EIF1AY
dtypes: float64(15748)
memory usage: 3.0 MB


In [17]:
early_group.describe()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
count,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,...,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000,25.000000
mean,2.510894,8.721781,18.531325,15.071854,24.152469,10.793160,9.009031,64.670644,37.296084,75.707467,...,5.720618,1.956668,2.075279,2.057353,1.794432,1.961449,17.567902,1.760707,2.151549,1.727730
std,1.469398,3.555897,5.269496,4.502477,4.958769,3.250598,3.146398,10.247193,6.753903,23.563379,...,2.340855,2.104235,2.247041,2.677120,1.351586,2.357573,6.103975,1.822367,3.029554,1.423561
min,0.885375,3.617088,11.410269,6.395270,12.343376,4.801919,4.082003,43.897364,24.616767,49.533508,...,2.080363,0.868346,0.868346,0.868346,0.868346,0.868346,6.839882,0.868346,0.868346,0.868346
25%,1.247440,6.375715,14.747056,12.727965,20.957007,9.050596,6.695258,59.082823,34.246019,62.319268,...,4.044178,1.001020,0.979074,0.979074,0.979074,0.979074,13.009698,0.979074,0.979074,0.979074
50%,2.131757,8.239048,17.985461,14.547256,24.190132,11.128772,8.766661,64.770521,36.349074,66.882473,...,5.850622,1.314098,1.247440,1.247440,1.264726,1.247440,17.618377,1.247440,1.247440,1.247440
75%,3.386331,10.578542,20.933827,17.237294,28.116276,13.546952,10.922912,71.335572,41.993794,76.227998,...,7.029290,1.817680,1.768454,1.647810,1.937270,1.647810,22.349725,1.647810,1.647810,1.768454
max,5.753165,16.458594,30.695485,24.620283,34.110743,16.377216,15.706400,85.701943,51.729112,128.606260,...,10.461652,9.927349,9.927349,10.945006,6.425951,10.361535,29.254009,9.702974,13.499597,7.078276


# Основні статистики хворої групи

In [18]:
cancer_group.head()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
49,1.618129,4.854387,7.646980,11.631036,19.839961,19.484855,15.492407,78.090449,49.035810,71.704607,...,7.175936,1.618129,1.618129,1.618129,1.618129,1.618129,22.008936,1.618129,1.618129,1.618129
50,1.609022,10.632542,28.344988,18.462903,23.702379,13.494061,6.091660,58.775216,88.026904,74.055824,...,6.091660,1.609022,1.609022,1.609022,1.609022,1.609022,24.840164,1.609022,1.609022,1.609022
51,1.549953,3.900629,12.496120,10.242207,22.972095,16.860563,7.324792,76.410218,109.988431,102.073552,...,6.873594,1.549953,1.549953,1.549953,1.549953,1.549953,18.834745,1.549953,1.549953,1.549953
52,1.606786,4.043656,8.849487,15.158388,26.287855,27.441093,8.033929,132.144503,188.797910,120.965118,...,1.606786,1.606786,1.606786,1.606786,1.606786,1.606786,21.215253,1.606786,1.606786,1.606786
53,2.697819,8.225604,21.299676,14.114099,22.289155,14.341455,5.567752,59.058349,103.291207,53.138337,...,7.099892,0.899273,0.899273,0.899273,0.899273,0.899273,20.368088,0.899273,0.899273,0.899273


In [19]:
cancer_group.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 49 to 71
Columns: 15748 entries, LOC643837 to EIF1AY
dtypes: float64(15748)
memory usage: 2.8 MB


In [20]:
cancer_group.describe()

,LOC643837,LOC100130417,SAMD11,NOC2L,KLHL17,PLEKHN1,C1orf170,HES4,ISG15,AGRN,...,CLIC2,RPS4Y1,ZFY,PRKY,USP9Y,DDX3Y,CD24,CYorf15B,KDM5D,EIF1AY
count,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,...,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000,23.000000
mean,2.186060,8.190456,23.692614,16.468034,25.035813,13.870940,9.677439,67.162763,91.706112,73.359613,...,4.684026,2.539207,2.168508,2.669397,1.871647,2.292908,15.752268,2.157294,2.776173,1.985272
std,1.257218,4.484989,12.648352,4.480521,3.181538,4.243144,3.357172,18.918964,66.516587,20.312533,...,2.030162,3.141084,2.523236,3.074528,2.101090,2.777428,6.640481,2.559228,4.278354,2.225919
min,0.833898,1.729668,7.646980,10.242207,19.839961,8.703401,5.567752,45.045398,27.944133,42.631422,...,1.176826,0.899273,0.899273,0.899273,0.833898,0.833898,4.675683,0.833898,0.899273,0.833898
25%,1.386976,5.045703,15.205378,13.842544,22.630625,11.443224,6.406841,54.126424,51.123794,56.948627,...,3.269521,1.044710,1.044710,1.044710,1.018990,1.018990,10.806873,1.018990,1.044710,1.018990
50%,1.609022,7.045005,21.331981,15.158388,24.408352,13.123224,10.317712,65.041865,63.205591,71.704607,...,4.578452,1.466240,1.428276,1.483635,1.176826,1.301619,13.978672,1.301619,1.428276,1.301619
75%,2.930559,10.582542,26.845599,18.045277,26.855281,16.190961,11.582120,73.684340,106.639819,84.521987,...,6.482627,1.613576,1.607904,2.289871,1.600338,1.607904,21.612094,1.600338,1.607904,1.600338
max,5.990041,20.006038,60.584449,29.659104,31.093687,27.441093,17.121366,132.144503,327.590426,120.965118,...,7.813974,11.913855,10.883310,11.692697,10.392539,10.640754,28.521715,9.919132,17.278985,9.333904


# Застосування критерія Стюдента для перевірки гіпотези про рівність середніх двух вибірок

## 1. Для груп normal (control) та early neoplasia (treatment)

In [98]:
result_ttest_normal_early = scipy.stats.ttest_ind(normal_group, early_group, equal_var=False)

In [99]:
match = {'Statistics': result_ttest_normal_early[0],
        'p-values': result_ttest_normal_early[1]}
result_ttest_normal_early = pd.DataFrame(match)

In [100]:
result_ttest_normal_early.head()

,Statistics,p-values
0,0.400289,0.690766
1,-4.608766,0.000032
2,-1.929277,0.060273
3,0.220542,0.826429
4,-2.013201,0.049876


In [101]:
answer_1 = len(result_ttest_normal_early[result_ttest_normal_early['p-values'] < 0.05])

In [102]:
write_answer('answer1.txt', answer_1)

## 2. Для групп early neoplasia (control) та cancer (treatment)

In [103]:
result_ttest_early_cancer = scipy.stats.ttest_ind(early_group, cancer_group, equal_var=False)

In [104]:
match = {'Statistics': result_ttest_early_cancer[0],
        'p-values': result_ttest_early_cancer[1]}
result_ttest_early_cancer = pd.DataFrame(match)

In [105]:
result_ttest_early_cancer.head()

,Statistics,p-values
0,0.824849,0.413735
1,0.452236,0.653429
2,-1.817266,0.079556
3,-1.075985,0.287581
4,-0.740329,0.463292


In [106]:
answer_2 = len(result_ttest_early_cancer[result_ttest_early_cancer['p-values'] < 0.05])

In [107]:
write_answer('answer2.txt', answer_2)

# Поправка методом Холма

## 1. Для груп normal (control) та early neoplasia (treatment)

In [161]:
reject, p_corrected, a1, a2 = smm.multipletests(result_ttest_normal_early['p-values'], 
                                            alpha = 0.05 / 2, 
                                            method = 'holm') 

In [162]:
result_ttest_normal_early['p_corrected'] = p_corrected
result_ttest_normal_early['reject'] = reject

In [163]:
result_ttest_normal_early.head()

,Statistics,p-values,p_corrected,reject
0,0.400289,0.690766,1.000000,False
1,-4.608766,0.000032,0.500174,False
2,-1.929277,0.060273,1.000000,False
3,0.220542,0.826429,1.000000,False
4,-2.013201,0.049876,1.000000,False


In [164]:
result_ttest_normal_early.reject.value_counts()

False    15746
True         2
Name: reject, dtype: int64

In [165]:
index_result_ttest_normal_early = result_ttest_normal_early.index[result_ttest_normal_early.reject == True]

In [166]:
fold_cahnges_normal_early = {}
for idx, i in enumerate(normal_group):
    T = np.mean(early_group[i])
    C = np.mean(normal_group[i])
    
    if T > C:
        fold_cahnges_normal_early[idx] = T/C
    else:
        fold_cahnges_normal_early[idx] = -C/T

In [167]:
answer_3 = 0
for idx in index_result_ttest_normal_early:
    if np.abs(fold_cahnges_normal_early[idx]) > 1.5:
        answer_3 += 1

In [168]:
write_answer('answer3.txt', answer_3)

## 2. Для групп early neoplasia (control) та cancer (treatment)

In [169]:
reject, p_corrected, a1, a2 = smm.multipletests(result_ttest_early_cancer['p-values'], 
                                            alpha = 0.05 / 2, 
                                            method = 'holm') 

In [170]:
result_ttest_early_cancer['p_corrected'] = p_corrected
result_ttest_early_cancer['reject'] = reject

In [171]:
result_ttest_early_cancer.head()

,Statistics,p-values,p_corrected,reject
0,0.824849,0.413735,1.0,False
1,0.452236,0.653429,1.0,False
2,-1.817266,0.079556,1.0,False
3,-1.075985,0.287581,1.0,False
4,-0.740329,0.463292,1.0,False


In [172]:
result_ttest_early_cancer.reject.value_counts()

False    15669
True        79
Name: reject, dtype: int64

In [173]:
index_result_ttest_early_cancer = result_ttest_early_cancer.index[result_ttest_early_cancer.reject == True]

In [174]:
fold_cahnges_early_cancer = {}
for idx, i in enumerate(early_group):
    T = np.mean(cancer_group[i])
    C = np.mean(early_group[i])
    
    if T > C:
        fold_cahnges_early_cancer[idx] = T/C
    else:
        fold_cahnges_early_cancer[idx] = -C/T

In [175]:
answer_4 = 0
for idx in index_result_ttest_early_cancer:
    if np.abs(fold_cahnges_early_cancer[idx]) > 1.5:
        answer_4 += 1

In [176]:
write_answer('answer4.txt', answer_4)

# Поправка методом Бенджаміні-Хохберга

## 1. Для груп normal (control) та early neoplasia (treatment)

In [177]:
reject, p_corrected, a1, a2 = smm.multipletests(result_ttest_normal_early['p-values'], 
                                            alpha = 0.05 / 2, 
                                            method = 'fdr_bh') 

In [178]:
result_ttest_normal_early['p_corrected'] = p_corrected
result_ttest_normal_early['reject'] = reject

In [179]:
result_ttest_normal_early.head()

,Statistics,p-values,p_corrected,reject
0,0.400289,0.690766,0.966511,False
1,-4.608766,0.000032,0.035698,False
2,-1.929277,0.060273,0.536103,False
3,0.220542,0.826429,0.980777,False
4,-2.013201,0.049876,0.499016,False


In [180]:
result_ttest_normal_early.reject.value_counts()

False    15744
True         4
Name: reject, dtype: int64

In [181]:
index_result_ttest_normal_early = result_ttest_normal_early.index[result_ttest_normal_early.reject == True]

In [182]:
answer_5 = 0
for idx in index_result_ttest_normal_early:
    if np.abs(fold_cahnges_normal_early[idx]) > 1.5:
        answer_5 += 1

In [184]:
write_answer('answer5.txt', answer_5)

## 2. Для групп early neoplasia (control) та cancer (treatment)

In [185]:
reject, p_corrected, a1, a2 = smm.multipletests(result_ttest_early_cancer['p-values'], 
                                            alpha = 0.05 / 2, 
                                            method = 'fdr_bh') 

In [186]:
result_ttest_early_cancer['p_corrected'] = p_corrected
result_ttest_early_cancer['reject'] = reject

In [187]:
result_ttest_early_cancer.head()

,Statistics,p-values,p_corrected,reject
0,0.824849,0.413735,0.675195,False
1,0.452236,0.653429,0.836406,False
2,-1.817266,0.079556,0.288873,False
3,-1.075985,0.287581,0.563007,False
4,-0.740329,0.463292,0.712214,False


In [188]:
result_ttest_early_cancer.reject.value_counts()

False    14916
True       832
Name: reject, dtype: int64

In [189]:
index_result_ttest_early_cancer = result_ttest_early_cancer.index[result_ttest_early_cancer.reject == True]

In [190]:
answer_6 = 0
for idx in index_result_ttest_early_cancer:
    if np.abs(fold_cahnges_early_cancer[idx]) > 1.5:
        answer_6 += 1

In [193]:
write_answer('answer6.txt', answer_6)